# AI Agents

## Basic prompting

In [1]:
import openai

openai_api_key = "PUT YOUR API KEY"
openai_client = openai.OpenAI(api_key=openai_api_key)

def get_completion(prompt, model="gpt-3.5-turbo"):
    chat_completion = openai_client.chat.completions.create(
        messages=[{"role": "user",
                   "content": prompt}],
        model=model,
    )
    return chat_completion.choices[0].message.content

In [4]:
get_completion("Tell me a joke in Slovak language.")

'Prečo sa veveričky nehodia do armády?\nPrečo?\nLebo majú oriešky!'

## Built-in LangChain tools

In [3]:
import langchain
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.chat_models import ChatOpenAI

import warnings

warnings.filterwarnings('ignore')

langchain.debug = True

In [5]:
# llm_model = "gpt-4o-mini"
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0, model=llm_model, openai_api_key=openai_api_key)
llm_4o = ChatOpenAI(temperature=0, model="gpt-4o", openai_api_key=openai_api_key)

Tools:
- functions that can be executed to help LLMs answer questions
- Examples: code execution, API requests, database querying, file system access

In [6]:
built_in_tools = load_tools(["llm-math", "wikipedia"], llm=llm)

In [7]:
agent = initialize_agent(
    built_in_tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True)

In [8]:
question_1 = "What is 1234 times 5678 ?"
agent(question_1)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is 1234 times 5678 ?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "What is 1234 times 5678 ?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

{'input': 'What is 1234 times 5678 ?', 'output': '7006652'}

In [11]:
get_completion(question_1)

'The product of 1234 and 5678 is 7,001,652.'

## Wikipedia example

In [12]:
question = "How many medals has Slovakia won during the Summer Olympics 2024 in Paris?"
result = agent(question)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How many medals has Slovakia won during the Summer Olympics 2024 in Paris?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "How many medals has Slovakia won during the Summer Olympics 2024 in Paris?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (w

In [13]:
get_completion(question)

"I'm sorry, but I cannot provide real-time updates on current events. Please check the official Olympics website or news sources for the most up-to-date information on Slovakia's medal count during the Summer Olympics 2024 in Paris."

## Python Agent

In [14]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [15]:
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff", "Fusion"],
                 ["Trance", "Former"],
                 ["Jen", "Ayai"]]

In [16]:
agent(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

Python REPL can execute arbitrary code. Use with caution.


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] [1.99s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "We can use the `sorted()` function in Python to sort the list of customers based on last name first and then first name.\nAction: Python_REPL\nAction Input: customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]\nsorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))\nprint(sorted_customers)",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "We can use the `sorted()` function in Python to sort the list o

{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"}

In [17]:
get_completion(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")

"[['Elle', 'Elem'], ['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

## Define your own tool

In [18]:
from langchain.agents import tool
from datetime import date
langchain.debug = True

In [19]:
@tool
def get_current_time() -> str:
    """Returns today's date, use this for any questions related to knowing today's date 
    or any requests where today's date is needed to be known. \
    The input should always be an empty string, and this function will always return 
    today's date - any date mathematics should occur outside this function."""
    return str(date.today())

In [20]:
agent = initialize_agent(
    built_in_tools + [get_current_time],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True)

In [21]:
result = agent("what's the date today?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "what's the date today?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "what's the date today?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., args: {'query': {'title': 'Query', 'description': 'query to look up on wikipedia', 'type': 'string'}}\nget_current_time: Returns today's date, use this for any 

In [22]:
get_completion("what's the date today?")

"I'm sorry, I am not able to provide real-time information. Please check your device or preferred source for today's date."

### Define HAPI tool

In [23]:
from pydantic import BaseModel, Field
from hapiclient import hapi
import numpy as np

# Define the input schema
class HapiToolInput(BaseModel):
    start: str = Field(..., description="Start timestamp to get the data in the format of YYYY-MM-DDTHH:MM:SS")
    stop: str = Field(..., description="End timestamp to get the data in the format of YYYY-MM-DDTHH:MM:SS")

@tool(args_schema=HapiToolInput)
def get_v_p_data_from_hapi(start: str, stop: str) -> dict:
    """Use this function to get some descriptive statistics for proton speed data (V_p parameter) of space weather within a given time range."""

    server = 'https://cdaweb.gsfc.nasa.gov/hapi'
    dataset = 'SOHO_CELIAS-PM_30S'
    parameter = 'V_p'
    opts = {'logging': False}

    v_p_data, meta_data = hapi(server, dataset, parameter, start, stop, **opts)
    v_p_vals = [data[1] for data in v_p_data]
    
    distro_params = {
        "average": np.mean(v_p_vals),
        "maximum": np.max(v_p_vals),
        "minimum": np.min(v_p_vals),
        "standard_deviation": np.std(v_p_vals)
    }
    
    return distro_params

In [24]:
agent = initialize_agent(
    built_in_tools + [get_v_p_data_from_hapi, get_current_time],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True)

In [25]:
result = agent("What was the average value of the V_p parameter during April of 2021?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What was the average value of the V_p parameter during April of 2021?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "What was the average value of the V_p parameter during April of 2021?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., args: {'query': {'title': 'Query', 'description': 'query to look u

In [26]:
result = agent("What was the maximum value of the V_p parameter in the first half of 2022?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What was the maximum value of the V_p parameter in the first half of 2022?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "What was the maximum value of the V_p parameter in the first half of 2022?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., args: {'query': {'title': 'Query', 'description': 'query

### More general HAPI tool

In [27]:
# Define the input schema
class HapiToolInput(BaseModel):
    start: str = Field(..., description="Start timestamp to get the data in the format of YYYY-MM-DDTHH:MM:SS")
    stop: str = Field(..., description="End timestamp to get the data in the format of YYYY-MM-DDTHH:MM:SS")
    dataset: str = Field(..., description="Name of the dataset to be used. It can be one of two values - "
                                          "'WI_H0_MFI@0' in case of the magnetic field vector data (parameter B_z) or "
                                          "'SOHO_CELIAS-PM_30S' in case of the proton speed data (parameter V_p)")

@tool(args_schema=HapiToolInput)
def get_v_p_b_z_data_from_hapi(start: str, stop: str, dataset: str) -> dict:
    """Use this function to get some descriptive statistics for either proton speed parameter (V_p) 
    or the magnetic field vector z-component (B_z) within a given time range."""

    server = 'https://cdaweb.gsfc.nasa.gov/hapi'
    if dataset == 'SOHO_CELIAS-PM_30S':
        parameter = 'V_p'
    elif dataset == 'WI_H0_MFI@0':
        parameter = 'BGSM'
    else:
        raise Exception("Unsupported dataset name!")
    
    opts = {'logging': False}

    data, meta_data = hapi(server, dataset, parameter, start, stop, **opts)
    vals = np.array([row[1] for row in data])
    if type(vals[0]) is np.ndarray:
        vals = vals[:, 2]

    distro_params = {
        "average": np.mean(vals),
        "maximum": np.max(vals),
        "minimum": np.min(vals),
        "standard_deviation": np.std(vals)
    }

    return distro_params

In [28]:
agent = initialize_agent(
    built_in_tools + [get_v_p_b_z_data_from_hapi, get_current_time],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True)

In [29]:
result = agent("What was the maximum value of the proton speed parameter in the first half of 2022?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What was the maximum value of the proton speed parameter in the first half of 2022?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "What was the maximum value of the proton speed parameter in the first half of 2022?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., args: {'query': {'title': 'Query', 'de

In [30]:
result = agent("What was the maximum value of the magnetic field parameter in the first half of 2022?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What was the maximum value of the magnetic field parameter in the first half of 2022?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "What was the maximum value of the magnetic field parameter in the first half of 2022?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., args: {'query': {'title': 'Query',

### Ask questions about Pandas DF

In [32]:
!pip3 install tabulate==0.9.0

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [42]:
import pandas as pd

df = pd.read_csv("swe_data_6parameters_2000-2023.csv")
df.head()

,date,f10,bz,dst,ap,pcn,wnm
0,2000-01-01 00:00:00,125.6,1.0,-45,56,3.7,111.823000
1,2000-01-01 01:00:00,125.6,-3.7,-37,56,2.2,111.616991
2,2000-01-01 02:00:00,125.6,-2.6,-37,56,4.4,111.398735
3,2000-01-01 03:00:00,125.6,-2.8,-41,39,6.3,111.736292
4,2000-01-01 04:00:00,125.6,-1.4,-45,39,3.7,111.547321


In [34]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True, allow_dangerous_code=True)

agent("how many rows are there?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "how many rows are there?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": ""
}


{'input': 'how many rows are there?', 'output': '210384'}

In [35]:
agent("When was the highest measured overall geomagnetic activity on Earth?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "When was the highest measured overall geomagnetic activity on Earth?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exitin

{'input': 'When was the highest measured overall geomagnetic activity on Earth?',
 'output': 'The highest measured overall geomagnetic activity on Earth occurred on the date: 2000-01-01 00:00:00'}

In [40]:
df_columns = [
    "timestamp",
    "solar_radio_flux",
    "interplanetary_magnetic_field",
    "geomagnetic_equatorial_index",
    "general_geomagnetic_level",
    "geomagnetic_activity_on_polar_caps",
    "weighted_neutron_monitor_index"
]
df.columns = df_columns

agent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True, allow_dangerous_code=True)
agent("When was the highest measured overall geomagnetic activity on Earth?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "When was the highest measured overall geomagnetic activity on Earth?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exitin

{'input': 'When was the highest measured overall geomagnetic activity on Earth?',
 'output': 'The highest measured overall geomagnetic activity on Earth was on 2000-01-01 03:00:00.'}

### Connect to DB

In [41]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine

In [43]:
# Path to your SQLite database file
database_file_path = "test.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')
df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False
)

210384

In [44]:
db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor_SQL = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    top_k=10,
    verbose=True,
    handle_parsing_errors=True
)

In [45]:
result = agent_executor_SQL("What is the minimum date for which we have data in the table?")

[chain/start] [chain:SQL Agent Executor] Entering Chain run with input:
{
  "input": "What is the minimum date for which we have data in the table?"
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:Ru

In [47]:
agent_executor_SQL("When was the lowest measured Dst?")

[chain/start] [chain:SQL Agent Executor] Entering Chain run with input:
{
  "input": "When was the lowest measured Dst?"
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Cha

{'input': 'When was the lowest measured Dst?',
 'output': 'The lowest measured Dst was -422 and it occurred on November 20, 2003 at 8:00 PM.'}

In [48]:
agent_executor_SQL("When was the Dst below -200 nT?")

[chain/start] [chain:SQL Agent Executor] Entering Chain run with input:
{
  "input": "When was the Dst below -200 nT?"
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain

{'input': 'When was the Dst below -200 nT?',
 'output': 'The Dst was below -200 nT on multiple occasions, including dates such as 2000-04-06, 2000-07-15, 2000-08-12, 2001-03-31, 2001-04-11, 2001-11-06, 2003-10-29, 2003-11-20, 2004-11-08, 2005-05-15, 2015-03-17, and 2023-04-24.'}